In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pandahouse as ph
import datetime
from datetime import timedelta
from operator import attrgetter
%matplotlib inline

## Какое количество пользователей, которые совершили покупку только один раз?

Для нашего решения, произведенная покупка - это статус delivered и время подтверждения оплаты order_approved_at из таблицы items, потому что на остальных этапах есть возможность отказаться от товара по любым причинам (например, передумал), а при статусе delivered товар может быть возвращен только из-за нарушения упаковки или не надлежащего вида изделия. Если коротко, то:

**Потребитель получил товар, продавец получил деньги.**

Узнаем кол-во покупотелей, совершивших только 1 покупку (иммеющих статус заказа delivered)

In [14]:
# Проверим общее количество уникальных покупателей
customers.customer_unique_id.nunique()

96096

In [15]:
# Теперь необходимо проверить, есть ли отличия в уникальности пользователей 2-х таблиц customers и items
print(customers.customer_id.nunique())
print(items.customer_id.nunique())
# Проверим общее количество уникальных идентификаторов заказа (номер чека)
items.order_id.nunique()

99441
99441


99441

In [16]:
# Соеденим customers и items
customers_items = customers.merge(items)
customers_items.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15


In [17]:
# Проверяем, есть ли изменения
print(customers_items.customer_unique_id.nunique())
print(customers_items.customer_id.nunique())
print(customers_items.order_id.nunique())

96096
99441
99441


In [18]:
customers_items.loc[customers_items.order_approved_at.notna()]\
                                   .query('order_status == "delivered"')\
                                   .groupby('customer_unique_id', as_index=False)\
                                   .agg({'order_id':'nunique'})\
                                   .query('order_id == 1')\
                                   .count()[0]
#1. Исключаем неоплаченные заказы
#2. Заказ доставлен
#3. Группируем по уникальному id
#4. Считаем уникальных заказов
#5. Был сделан только 1 заказ
#6. Кол-во пользователей (для минимализма)

90545

 **Ответ: 90545 пользователей совершили покупку 1 раз**